# D - classification of change dates

**REQUIREMENT: (A), (B) and (C)**

This program carries out the classification of the changement detected by program (B) using the following function:

> **dates_change_classification(sites, dteChgEnd, dteExe)**

This function operates the following step:

**1) Creation/Determination of the result table**

This table is named **classif_bimestrial_chg_dates**

**2) For each site, reseach of change date potentially detected by script (B)**

Change dates are stored in table named as following '{0}_dates_{1}'.format(site, dteExe).

If there is no change date for a site, the related table is just empty.

**3) For each site, for each of its change date, calculation of relevant periods of investigation from change dates**

For S1 indexes (VH), their average over ((change date) - (change date + 30 days)) will be compared to their average over the same period the year before (((change date) - (change date + 30 days)) - 1 year).

For S2 indexes (NDVI, BAI), their average over ((change date) - (change date + 60 days)) will be compared to their average over the same period the year before (((change date) - (change date + 60 days)) - 1 year).


**4) For each site, for each of its change date, calculation of index averages over these relevant periods and their amplitude**

For a given index, its amplitude is its variation (difference) between the "current" period and the same period the year before.

**5) For each site, for each of its change date, calculation of the number of S2 which were exploitable over these relevant periods**

This information is later used for the assessment of change confidence (G).


**6) For each site, for each of its change date, caracterisation of changes based on index averages**

Vegetation change caracterisation is based on NDVI analysis.

Soil change caracterisation is based on BAI analysis.

Building change caracterisation is based on VH analysis.

**Code interpretation key**

> 0 No change

> 1 Change (no additional information)

> 2 Increase

> 3 Decrease

> 4 Possible change (no additional information)

> 5 Possible increase

> 6 Possible decrease


## Import libraries and functions

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # Localisation of SARSAR libraries

#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/")                   # emplacement des modules RenPri
#sys.path.append("/home/issep/sarsar-issep/SARSAR_utils/rme_chg_detection_module/") # emplacement de la fonction de Mattia

from issep import sarsar_admin
from os.path import join
from lecture_ini import config
from select_sites import sites_to_process

## Definition of the **change_date_classfication** function

In [ ]:
# Defines a function to convert formatted strings into date objects; on which intervals can be calculated.
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

In [ ]:
def change_date_classification(sites, dteChgEnd, dteExe):
    """ This function classifies/determines the type of change which occurs at the detected change dates, either vegetation, soil or building change.
        Results of this function are returned in the following table: "classif_bimestrial_chg_dates".
        
            Parameters (they are automatically read in "sarsar.ini" by the function "config"):
                sites (list): list of sites (sar_id_segment) to process
                dteChgEnd (date): maximum change date (YYYY-MM-DD)
                dteExe: processing/execution date (YYYYMMDD)
                """
    ##READING PARAMETERS IN SARSAR.IN with CONFIG
    
    dates = config(section='dates')
    nbrDays_S1 = int(dates['days_s1'])
    nbrDays_S2 = int(dates['days_s2'])
    
    ## CONNECTION TO DB
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)

    conn = sarsar_admin._create_or_get_db_connection()
    cur = None

    ## STARTING CHANGE DATE CLASSIFICATION

    try:
        import psycopg2.extras
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)
        cur2 = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

    ## (RE-)CREATION OF THE RESULT TABLE

        table_name = 'classif_bimestrial_chg_dates'

        strSQL = 'CREATE TABLE IF NOT EXISTS "{0}" (ID_Segment TEXT, dteExe DATE, dteChgEnd DATE, date DATE, NDVI_amplitude NUMERIC(4,3), BAI_amplitude NUMERIC(4,3), VH_amplitude NUMERIC(4,3), vegetation INTEGER, soil INTEGER, building INTEGER, nImages_a INTEGER, nImages_p INTEGER);'.format(table_name)
        cur.execute(strSQL)

#         # Liste les sar_id_segment pour lesquels j'ai des observations dans l'intervalle de dates
#         strSQL = '''SELECT DISTINCT sar_id_segment 
#                     FROM sar_index_stats WHERE index_name NOT IN ('BI2','VV','SBI','NDVI','BI2_part1','BI','BAI','VH') 
#                     AND substring(index_name,1,2) != 'VV' AND acq_date BETWEEN '{0}' AND '{1}' 
#                     ORDER BY sar_id_segment;'''.format(dteDebut, dteFin)
#         cur.execute(strSQL)
#         sites = [item[0] for item in cur.fetchall()]

        # DEBUG: force la liste
    #        sites = ['62003-ISA-0007-01', '62063-ISA-0073-01', '62096-ISA-0056-01','52012-ISA-0010-01']

    #    i_debug = 0               # DEBUG
    #    sites = sites[i_debug:]   # DEBUG
    

    ## FETCHING CHANGE DATES RETURNING BY CHANGE DETECTION (B)
        for site in sites:
    #        i_debug += 1
    #        print(site, i_debug)  # DEBUG

            strSQL = 'SELECT * FROM "{0}" ORDER BY change_date;'.format('{0}_dates_{1}'.format(site, dteExe))
            cur.execute(strSQL)
            
    ## INDEX AMPLITUDE CALCULATION
            # If there is at least one change date
            if cur.rowcount > 0:

                # Fetchall is converted into a list
                dates = [item[0] for item in cur.fetchall()]  # .strftime("%Y-%m-%d")

                # List is converted into a dataframe (df)
                listeDates = pd.DataFrame (dates, columns = ['Change date'])
    #            print(listeDates)  # DEBUG

                # Calculation of periods over which index amplitude will be calculted
                # Period limits are added to the df in new columns
                listeDates['a_debut_S2'] = listeDates['Change date']
                listeDates['a_fin_S2'] = listeDates['Change date'] + datetime.timedelta(days = nbrDays_S2)
                listeDates['p_debut_S2'] = listeDates['a_debut_S2'] - datetime.timedelta(days = 365)
                listeDates['p_fin_S2'] = listeDates['a_fin_S2'] - datetime.timedelta(days = 365)

                listeDates['a_debut_S1'] = listeDates['Change date']
                listeDates['a_fin_S1'] = listeDates['Change date'] + datetime.timedelta(days = nbrDays_S1)
                listeDates['p_debut_S1'] = listeDates['a_debut_S1'] - datetime.timedelta(days = 365)
                listeDates['p_fin_S1'] = listeDates['a_fin_S1'] - datetime.timedelta(days = 365)

                # For each change date, check that the smoothed data for each indice is available; 
                # and calculate amplitude as the difference between index averaged value over the a-period compared to its value over the same period a year earlier (p-period).
                
                # Initialisation of variabkes
                for i in listeDates.index:
                    NDVI_amplitude = None
                    BAI_amplitude = None
                    VH_amplitude = None
                    vegetation = None
                    soil = None
                    building = None

    # NDVI__________________
                    # Fetch NDVI smoothed table in DB
                    table_smooth = '{0}_NDVI_{1}_smoothed'.format(site, dteExe)

                    # Check if table exists
                    strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                    cur2.execute(strSQL)

                    # If table exists and contains values, calculate NDVI amplitude
                    NDVI_EXISTS = cur2.fetchone()[0]

                    if NDVI_EXISTS == True:
                        strSQL = 'SELECT a.moyenne, p.moyenne as moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg FROM (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{1}\' AND \'{2}\') a, (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{3}\' AND \'{4}\') p;'.format(table_smooth, listeDates['a_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S2'][i].strftime('%Y-%m-%d'), listeDates['p_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S2'][i].strftime('%Y-%m-%d'))
                        cur2.execute(strSQL)
                        NDVI_amplitude = cur2.fetchone()['moyenne_chg']

    # BAI__________________
                    # Fetch NDVI smoothed table in DB
                    table_smooth = '{0}_BAI_{1}_smoothed'.format(site, dteExe)

                    # Check if table exists
                    strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                    cur2.execute(strSQL)

                    # If table exists and contains values, calculate BAI amplitude
                    BAI_EXISTS = cur2.fetchone()[0]

                    if BAI_EXISTS == True:
                        strSQL = 'SELECT a.moyenne, p.moyenne as moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg FROM (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{1}\' AND \'{2}\') a, (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{3}\' AND \'{4}\') p;'.format(table_smooth, listeDates['a_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S2'][i].strftime('%Y-%m-%d'), listeDates['p_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S2'][i].strftime('%Y-%m-%d'))
                        print(strSQL)
                        cur2.execute(strSQL)
                        BAI_amplitude = cur2.fetchone()['moyenne_chg']

    # VH__________________
                    # Fetch NDVI smoothed table in DB
                    table_smooth = '{0}_VH_{1}_smoothed'.format(site, dteExe)

                    # Check if table exists
                    strSQL = "SELECT EXISTS (SELECT * FROM information_schema.tables WHERE table_name = '{0}');".format(table_smooth)
                    cur2.execute(strSQL)

                    # If table exists and contains values, calculate VH amplitude
                    VH_EXISTS = cur2.fetchone()[0]

                    if VH_EXISTS == True:
                        strSQL = 'SELECT a.moyenne, p.moyenne as moyenne_p, ROUND(a.moyenne-p.moyenne, 3) AS moyenne_chg FROM (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{1}\' AND \'{2}\') a, (SELECT avg(indice) as moyenne FROM "{0}" WHERE dte BETWEEN \'{3}\' AND \'{4}\') p;'.format(table_smooth, listeDates['a_debut_S1'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S1'][i].strftime('%Y-%m-%d'), listeDates['p_debut_S1'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S1'][i].strftime('%Y-%m-%d'))
                        cur2.execute(strSQL)
                        VH_amplitude = cur2.fetchone()['moyenne_chg']


    ## CALCULATED S2 IMAGES AVAILABILITY
    
                    nImages_a = 0
                    nImages_p = 0

                    strSQL = 'SELECT count(*) FROM sar_index_stats WHERE sar_id_segment = \'{0}\' AND index_name = \'NDVI\' AND acq_date BETWEEN \'{1}\' AND \'{2}\';'.format(site, listeDates['a_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['a_fin_S2'][i].strftime('%Y-%m-%d'))
                    cur2.execute(strSQL)
                    nImages_a = cur2.fetchone()[0]

                    strSQL = 'SELECT count(*) FROM sar_index_stats WHERE sar_id_segment = \'{0}\' AND index_name = \'NDVI\' AND acq_date BETWEEN \'{1}\' AND \'{2}\';'.format(site, listeDates['p_debut_S2'][i].strftime('%Y-%m-%d'), listeDates['p_fin_S2'][i].strftime('%Y-%m-%d'))
                    cur2.execute(strSQL)
                    nImages_p = cur2.fetchone()[0]


    ## CARACTERIZATION OF CHANGE DATE

                    # Vegetation change
                    if NDVI_amplitude != None:
                        if NDVI_amplitude >= 0.1:
                            vegetation = 2  # « Increase in vegetation »

                        elif NDVI_amplitude <= -0.1:
                            vegetation = 3  # « Decrease in vegetation  »

                        else:
                            vegetation = 0  # « No change in vegetation »

                    # Soil change
                    if BAI_amplitude != None:

                        if abs(BAI_amplitude) >= 0.05:
                            soil = 1  # « Change in soil »

                        else:
                            soil = 0  # « No change in soil »

                    # Building change
                    if VH_amplitude != None:
                        if VH_amplitude >= 0.135:
                            building = 2  # « Increase in building »

                        elif VH_amplitude <= -0.135:
                            building = 3  # « Decrease in building »

                        else:
                            building = 0  # « No change in building »
                            
    ## SAVING RESULTS
                    # Inserting results in the result table

                    strSQL = '''INSERT INTO {0} (ID_Segment, dteExe, dteChgEnd, date, NDVI_amplitude, BAI_amplitude, VH_amplitude, vegetation, soil, building, nImages_a, nImages_p) 
                                VALUES (\'{1}\', \'{2}\', \'{3}\', \'{4}\', {5}, {6}, {7}, {8}, {9}, {10}, \'{11}\', \'{12}\');'''.format(table_name, site, f'{dteExe[0:4]}-{dteExe[4:6]}-{dteExe[6:]}', dteChgEnd, listeDates['Change date'][i].strftime('%Y-%m-%d'), NDVI_amplitude, BAI_amplitude, VH_amplitude, vegetation, soil, building, nImages_a, nImages_p)
    #                print(strSQL)
                    cur2.execute(strSQL)

                    # Posting all modifications
                    conn.commit()

        cur2.close()
        cur.close()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        
    finally:
        if cur2 is not None:
            cur2.close()

        if cur is not None:
            cur.close()
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()

## Calling functions

In [ ]:
# Call config
dates = config(section='dates')
dteDebut = dates['deb']
dteFin = dates['fin']
dteExe = dates['exe']
dteChgEnd = dates['chg_end']

## Call sites_to_process
lstSARs = sites_to_process(dteDebut, dteFin)

## Call dates_change_classification
change_date_classification(lstSARs, dteChgEnd, dteExe)